<a href="https://colab.research.google.com/github/SilasBereit/turbomachinery-pumpenanalyse/blob/main/pump_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import requests

# URL der CSV-Datei auf GitHub (ersetze dies mit der tatsächlichen URL deiner Datei)
url = 'https://raw.githubusercontent.com/USERNAME/REPOSITORY/main/volume_flow_data.csv'

# Die CSV-Datei direkt von GitHub in ein DataFrame laden
df = pd.read_csv(url)

# Spalten benennen (falls keine Header)
df.columns = ['timestamp', 'flow_rate_m3h']

# Datentypen umwandeln
df['timestamp'] = pd.to_datetime(df['timestamp'])

# m³/h → m³/s
df['flow_rate_m3s'] = df['flow_rate_m3h'] / 3600

# Zeitdifferenz berechnen
df['time_delta_s'] = df['timestamp'].diff().dt.total_seconds()
df['time_delta_s'].fillna(0, inplace=True)

# Kennlinie für Impeller Ø264 mm (geschätzt aus Diagramm)
flow_m3h = np.array([0, 50, 100, 150, 200, 250, 300, 350, 400])
head_m = np.array([44, 43, 41, 38, 34, 29, 24, 18, 12])

# Interpolation
head_interpolator = interp1d(flow_m3h, head_m, bounds_error=False, fill_value="extrapolate")
df['head_m'] = head_interpolator(df['flow_rate_m3h'])

# Konstanten
rho = 1000       # Dichte [kg/m³]
g = 9.81         # Erdbeschleunigung [m/s²]

# Hydraulische Leistung
df['P_hydraulic'] = rho * g * df['flow_rate_m3s'] * df['head_m']  # [Watt]

# Energie
df['E_hydraulic'] = df['P_hydraulic'] * df['time_delta_s']

# Gesamtenergie in kWh
total_energy_joule = df['E_hydraulic'].sum()
total_energy_kWh = total_energy_joule / 3.6e6

print(f"Gesamte hydraulische Energie: {total_energy_kWh:.2f} kWh")



Saving volume_flow_data.csv to volume_flow_data (1).csv
Gesamte hydraulische Energie: 412.70 kWh


<ipython-input-4-c7cd3545e208>:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['time_delta_s'].fillna(0, inplace=True)
